# Clothing Segmentation using U-Net Model

### 1. Import the Libraries

In [ ]:
# Importing the libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### 2. Load the Data

#### 2.1 Create DataLoader Class

In [ ]:
# Class to load the dataset, convert the images and masks to tensors, 
# normalize the images, resize the images and masks, and split the dataset 
# into train and validation
class DatasetLoader():
    def __init__(self):
      pass

    '''
      Load images and masks from the dataset

      @param images_path - path to the images
      @param masks_path - path to the masks
      @return train_dataset - train dataset
      @return validation_dataset - validation dataset
    '''
    def load_images_and_masks(self, images_path, masks_path):
      image_paths = []
      mask_paths = []

      # Put the images paths into a list
      for root, dir, files in os.walk(images_path):
        for file in files:
          path = os.path.join(root, file)
          image_paths.append(path)

      # Put the masks paths into a list
      for root, dir, files in os.walk(masks_path):
        for file in files:
          path = os.path.join(root, file)
          mask_paths.append(path)

      # Sort the lists into their correct orders
      image_paths.sort()
      mask_paths.sort()

      # Check the length of images and masks
      print("Number of images: ", len(image_paths))
      print("Number of masks: ", len(mask_paths))
      
      # Convert images and masks to tensors
      images, masks = self._convert_images_masks_to_tensors(image_paths, mask_paths)

      # Normalize images
      images = self._normalize_images(images)

      # Resize images and masks
      images, masks = self._resize_images_and_masks(images, masks)

      # Split dataset into train and validation
      train_dataset, validation_dataset = self._train_test_split_dataset(images, masks)

      return train_dataset, validation_dataset

    '''
      Convert images and masks to tensors

      @param image_paths - list of image paths
      @param mask_paths - list of mask paths
      @return images - list of images
      @return masks - list of masks
    '''
    def _convert_images_masks_to_tensors(self, image_paths, mask_paths):
      images = []
      masks = []

      # Convert images to tensors
      for image in image_paths:
        image = tf.io.read_file(image)
        image = tf.image.decode_png(image, channels=3, dtype=tf.uint8)
        images.append(image)

      # Convert masks to tensors
      for mask in mask_paths:
        mask = tf.io.read_file(mask)
        mask = tf.image.decode_png(mask, channels=1, dtype=tf.uint8)
        masks.append(mask)

      return images, masks

    '''
      Split dataset into train and validation

      @param images - list of images
      @param masks - list of masks
      @return train_dataset - train dataset
      @return validation_dataset - validation dataset
    '''
    def _train_test_split_dataset(self, images, masks):
      train_X, validation_X, train_y, validation_y = train_test_split(images, masks, test_size=0.2, random_state=42)

      # Convert train and validation datasets to tensors
      train_X = tf.data.Dataset.from_tensor_slices(train_X)
      train_y = tf.data.Dataset.from_tensor_slices(train_y)

      # Convert the validation datasets to tensors
      validation_X = tf.data.Dataset.from_tensor_slices(validation_X)
      validation_y = tf.data.Dataset.from_tensor_slices(validation_y)

      # create a 1-1 mapping from X -> y
      train_dataset = tf.data.Dataset.zip(train_X, train_y)
      validation_dataset = tf.data.Dataset.zip(validation_X, validation_y)

      return train_dataset, validation_dataset

    '''
      Normalize input images

      @param images - list of images
      @return image_list - list of normalized images
    '''
    def _normalize_images(self, images):
      image_list = []
      for image in images:
        image = tf.cast(image, tf.float32)
        image = image / 255.0
        image_list.append(image)

      return image_list

    '''
      Resize images and masks

      @param images - list of images
      @param masks - list of masks
      @return image_list - list of resized images
      @return mask_list - list of resized masks
    '''
    def _resize_images_and_masks(self, images, masks):
      image_list = []
      mask_list = []

      for image in images:
        image = tf.image.resize(image, (384, 384))
        image_list.append(image)

      for mask in masks:
        mask = tf.cast(mask, tf.uint8)
        mask = tf.image.resize(mask, (384, 384),
                                      method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        mask_list.append(mask)

      return image_list, mask_list

#### 2.2 Load, Normalize and Resize image and masks

In [ ]:
# Define dataset paths
TRAIN_DATASET_PATH = "Resources/images"
VALIDATION_DATASET_PATH = "Resources/masks"

In [ ]:
# Create a DatasetLoader object
dataset_loader = DatasetLoader()

In [ ]:
# Load the data set, convert the images and masks to tensors, normalize the images,
# resize the images and masks, and split the dataset into train and validation
train_dataset, validation_dataset = dataset_loader\
    .load_images_and_masks(TRAIN_DATASET_PATH, VALIDATION_DATASET_PATH)

In [ ]:
# Chek that the train and validation datasets are loadaed correctly
print("Training Dataset #:", train_dataset.element_spec)
print("Training Dataset #:", validation_dataset.element_spec)

In [ ]:
# Check lentgh of train and validation datasets
print("Length of train dataset: ", len(train_dataset))
print("Length of validation dataset: ", len(validation_dataset))

### 3. Verify the loaded data and masks

#### 3.1 Create ImageViewer Class

In [ ]:
# Class to vie the images and masks
class ImageViewer():
    def __init__(self):
      pass

    '''
      Show images and masks from a dataset

      @param dataset - dataset to containg images and masks to display
      @param num_to_show - an even number of images to show (default is 2)
    '''
    def show_from_dataset(self, dataset, num_to_show=2):

      dataset = dataset.shuffle(100)

      for image, mask in dataset.take(5):
        plt.figure(figsize=(15,15))
        for i in range(num_to_show):
          if (i % 2) == 0:
            plt.subplot(1,2,i+1)
            plt.imshow(image)
            plt.axis('off')
          else:
            plt.subplot(1,2,i+1)
            plt.imshow(mask, cmap='plasma')
            plt.axis("off")
            plt.show()

    '''
      Plot two images side-by-side
      @param image1 -image 1
      @param image2 - image 2
    '''
    def plot_two_images(self, image1, image2):
      plt.figure(figsize=(15,15))
      plt.subplot(1,2, 1)
      plt.imshow(image1)
      plt.axis('off')

      plt.subplot(1,2, 2)
      plt.imshow(image2)
      plt.axis("off")
      plt.show()

    '''
      Show prediction

      @param image - image
      @param mask - mask
      @param pred - prediction
    '''
    def show_pred(self, image, mask, pred):
        plt.figure(figsize=(20,28))

        k = 0
        for i in pred:
            # show the predicted mask
            plt.subplot(4,3,1+k*3)
            i = tf.argmax(i, axis=-1)
            plt.imshow(i, cmap='plasma')
            plt.axis('off')
            plt.title('Prediction')

            # show the groundtruth mask
            plt.subplot(4,3,2+k*3)
            plt.imshow(mask[k], cmap='plasma')
            plt.axis('off')
            plt.title('Ground Truth')

            # show the real image
            plt.subplot(4,3,3+k*3)
            plt.imshow(image[k])
            plt.axis('off')
            plt.title('Actual')
            k += 1
            if k == 4: break

        plt.suptitle('Preditions', color='blue', size=24)
        plt.show()

#### 3.2 View Image and correspoding mask

In [ ]:
# Create an image viewer object
image_viewer = ImageViewer()
# View the images and masks from the train_dataset
image_viewer.show_from_dataset(train_dataset)

### 4. Augment the Images and Masks

#### 4.1 Create an AugmentDataset Class

In [ ]:
class AugmentDataset():
    def __init__(self):
      pass

    '''
      Augment dataset

      @param image_dataset - image dataset
      @return image_dataset - augmented image dataset
    '''
    def augment_dataset(self, image_dataset):
      a = image_dataset.map(self._brightness)
      b = image_dataset.map(self._contrast)
      c = image_dataset.map(self._saturation)
      d = image_dataset.map(self._hue)
      e = image_dataset.map(self._crop)
      f = image_dataset.map(self._flip)
      g = image_dataset.map(self._rotate)
      h = image_dataset.map(self._rotate270)
      i = image_dataset.map(self._flip_up_down)
      j = image_dataset.map(self._gamma)

      # Concatenate all augmented datasets to return a dataset 7X larger
      image_dataset = image_dataset.concatenate(a)
      image_dataset = image_dataset.concatenate(b)
      image_dataset = image_dataset.concatenate(c)
      image_dataset = image_dataset.concatenate(d)
      image_dataset = image_dataset.concatenate(e)
      image_dataset = image_dataset.concatenate(f)
      image_dataset = image_dataset.concatenate(g)
      image_dataset = image_dataset.concatenate(h)
      image_dataset = image_dataset.concatenate(i)
      image_dataset = image_dataset.concatenate(j)

      return image_dataset

    '''
      Adjust an image's brightness

      @param image - image
      @return bright adjusted image
    '''
    def _brightness(self, image, mask):
        image = tf.image.adjust_brightness(image, 0.1)
        return image, mask

    '''
      Adjust an image and mask's contrast

      @param image - image
      @param mask - mask
      @return contrast adjusted image
      @return contrast adjusted mask
    '''
    def _contrast(self, image, mask):
        image = tf.image.adjust_contrast(image, 0.1)
        return image, mask

    '''
      Adjust an image and mask's saturation

      @param image - image
      @param mask - mask
      @return saturation adjusted image
      @return saturation adjusted mask
    '''
    def _saturation(self, image, mask):
        image = tf.image.adjust_saturation(image, 0.1)
        return image, mask

    '''
      Adjust an image and mask's hue

      @param image - image
      @param mask - mask
      @return hue adjusted image
      @return hue adjusted mask
    '''
    def _hue(self, image, mask):
        image = tf.image.adjust_hue(image, 0.1)
        return image, mask

    '''
      Crop an image and mask

      @param image - image
      @param mask - mask
      @return cropped image
      @return cropped mask
    '''
    def _crop(self, image, mask):
      tf.image.central_crop(image, 0.7)
      tf.image.resize(image, (384, 384))

      mask = tf.image.central_crop(mask, 0.7)
      mask = tf.image.resize(mask, (384,384))
      mask = tf.cast(mask, tf.uint8)

      return image, mask

    '''
      Flip an image and mask

      @param image - image
      @param mask - mask
      @return flipped image
      @return flipped mask
    '''
    def _flip(self, image, mask):
      image = tf.image.flip_left_right(image)
      mask = tf.image.flip_left_right(mask)
      return image, mask

    '''
      Rotate an image and mask 90 degress

      @param image - image
      @param mask - mask
      @return rotated image
      @return rotated mask
    '''
    def _rotate(self, image, mask):
      image = tf.image.rot90(image)
      mask = tf.image.rot90(mask)
      return image, mask
    
    '''
      Rotate an image and mask 270 degrees

      @param image - image
      @param mask - mask
      @return rotated image
      @return rotated mask
    '''
    def _rotate270(self, image, mask):
      image = tf.image.rot90(image, -1)
      mask = tf.image.rot90(mask, -1)
      return image, mask
    
    '''
      Flip an image and mask upside down

      @param image - image
      @param mask - mask
      @return flipped image
      @return flipped mask
    '''
    def _flip_up_down(self, image, mask):
      image = tf.image.flip_up_down(image)
      mask = tf.image.flip_up_down(mask)
      return image, mask
    
    '''
      Adjust an image and mask's gamma

      @param image - image
      @param mask - mask
      @return gamma adjusted image
      @return gamma adjusted mask
    '''
    def _gamma(self, image, mask):
        image = tf.image.adjust_gamma(image, 0.1)
        return image, mask

#### 4.2 Augment the images and masks

In [ ]:
# Create an AugmentDataset object
augment_dataset = AugmentDataset()
# Augment the train_dataset
train_dataset = augment_dataset.augment_dataset(train_dataset)

In [ ]:
# Print the length of the train_dataset after augmentation
print("Validation Dataset: ", len(validation_dataset))
print("Training Dataset: ", len(train_dataset))

In [ ]:
# Print the element spec of the train_dataset to check if the augmentation was successful
print("Training Dataset #:", train_dataset.element_spec)

### 5. U-Net Model

#### 5.1 Define Model Parameters

In [ ]:
# Model Parameters
BATCH = 64
AT = tf.data.AUTOTUNE
BUFFER = 1000

STEPS_PER_EPOCH = 800//BATCH
VALIDATION_STEPS = 200//BATCH
INITIAL_EPOCHS = 2
FINE_TUNE_EPOCHS = 150

#### 5.2 Create Model Builder Class

In [ ]:
# Model Builder Class
class ModelBuilder():
    def __init__(self):
      self.down_sampling_model = None
      self.unet_model = None

    '''
      Create a unet model

      @return unet model
    '''
    def build_unet_model(self):
      # create the upsampling model using a combination of upsampling models
      upsampling_model = [
          self.build_upsampling_model(512, 4),
          self.build_upsampling_model(256, 4),
          self.build_upsampling_model(128, 4),
          self.build_upsampling_model(64, 4)
      ]

      # create the input layer using the shape of the images
      inputs = tf.keras.layers.Input(shape=[384, 384, 3])

      self.down_sampling_model = self.build_down_sampling_model()

      # store the results of the inputs going through the downsampling model
      down_sample = self.down_sampling_model(inputs)

      # store the last layer result as an output
      output = down_sample[-1]

      # store every output from the downsampling layer except the last one
      skips = reversed(down_sample[:-1])

      # loop through the upsampling model and the skips and concatenate them
      for upsample, skip in zip(upsampling_model, skips):
        output = upsample(output)
        output = tf.keras.layers.Concatenate()([output, skip])

      # downsample the output to get 59 mask potential values
      output = tf.keras.layers.Conv2DTranspose(
          filters=59,
          kernel_size=3,
          strides=2,
          padding='same')(output)

      # create and compile the UNET model
      self.unet_model = tf.keras.Model(inputs=inputs, outputs=output)
      self.compile_unet_model()

      return self.unet_model

    '''
      Compile a unet model
    '''
    def compile_unet_model(self):
      self.unet_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
        metrics=['accuracy'])

    '''
      Create a fine-tuned unet model

      @return fine-tuned unet model
    '''
    def build_fine_tuned_unet_model(self):
      # enable training for the down_sampling_model (MobileNetV2)
      self.down_sampling_model.trainable = True

      # recompile the model
      self.compile_unet_model()

      return self.unet_model

    '''
      Create a down-sampling model

      @return down-sampling model
    '''
    def build_down_sampling_model(self):
      # using MobileNetV2 but any other choice could probably work here
      base_model = tf.keras.applications.MobileNetV2(input_shape=(384, 384, 3), include_top=False, weights="imagenet")

      # Looking through the MobileNetV2 plot_model I found these blocks that
      # fit the needs of the down_sampling_model.
      skip_layer_names = [
          "block_1_expand_relu", # 128 -> 64
          "block_2_expand_relu", # 64 -> 32
          "block_5_expand_relu", # 32 -> 16
          "block_8_expand_relu", # 16 -> 8
          "block_14_expand_relu", # 8 -> 4
      ]

      # get the skip layers from the base model using the layer names
      skip_layers = [base_model.get_layer(name).output for name in skip_layer_names]

      # create the down_sampling model with the base model having training disabled
      down_stack = tf.keras.Model(inputs=base_model.input, outputs=skip_layers)
      down_stack.trainable = False

      return down_stack

    '''
      Create an up-sampling model

      @param filters - number of filters
      @param size - size of the kernel
      @return up-sampling model
    '''
    def build_upsampling_model(self, filters, size):
      initializer = tf.random_normal_initializer(0., 0.02)

      upsampling_model = tf.keras.Sequential()
      upsampling_model.add(tf.keras.layers.Conv2DTranspose(
          filters, size, strides=2,
          padding='same',
          kernel_initializer=initializer,
          use_bias=False))

      upsampling_model.add(tf.keras.layers.BatchNormalization())
      upsampling_model.add(tf.keras.layers.ReLU())

      return upsampling_model

### 5.3 Build the U-Net Model

In [ ]:
# Create a ModelBuilder object
model_builder = ModelBuilder()
# Build a U-Net model
unet_model = model_builder.build_unet_model()
# Print the summary of the U-Net model
unet_model.summary()

In [ ]:
# Cache the dataset in memory, shuffle the dataset, batch the dataset into mini-batches and
# repeat the dataset (this is used to ensure that the model sees the entire dataset)
train_dataset = train_dataset.cache().shuffle(BUFFER).batch(BATCH).repeat()

In [ ]:
# Prefetch - creates a background thread that preloads data while the model is training
train_dataset = train_dataset.prefetch(buffer_size=AT)

In [ ]:
# Batch the validation dataset - This function splits the validation dataset 
# into mini-batches of size BATCH.
validation_dataset = validation_dataset.batch(BATCH)

In [ ]:
# Infer on train dataset
example = next(iter(train_dataset))
preds = unet_model(example[0])
pred_mask = tf.argmax(preds, axis=-1)
pred_mask = tf.expand_dims(pred_mask, -1)

pred_mask = pred_mask[0]
actual_image = example[0][0]

image_viewer.plot_two_images(actual_image, pred_mask)

#### 5.4 Train/Fit the model

In [ ]:
# Train the model
history = unet_model.fit(
    train_dataset,
    epochs=INITIAL_EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    validation_data=validation_dataset
)

#### 5.5 Check the prediction

In [ ]:
# Use the model to make a prediction
image, mask = next(iter(validation_dataset))
pred = unet_model.predict(image)

In [ ]:
# View the image, mask, and prediction
image_viewer.show_pred(image, mask, pred)

#### 5.5 Model Performance

In [ ]:
class PerformanceReport():
    def __init__(self):
      self.conf_matrix = None

    '''
      Plot the UNET model's loss performance

      @param model_history - the unet model history
    '''
    def plot_loss_performance_charts(self, model_history):
      loss = model_history.history['loss']
      val_loss = model_history.history['val_loss']

      plt.figure()
      plt.plot(model_history.epoch, loss, 'r', label='Training loss')
      plt.plot(model_history.epoch, val_loss, 'bo', label='Validation loss')
      plt.title('Training and Validation Loss')
      plt.xlabel('Epoch')
      plt.ylabel('Loss Value')
      plt.ylim([0.5, 1])
      plt.legend()
      plt.show()
    
    '''
      Plot the UNET model's loss performance - with ploty

      @param model_history - the unet model history
    '''
    def plot_loss_performance_charts_plotly(self, model_history):
      loss = model_history.history['loss']
      val_loss = model_history.history['val_loss']

      fig = go.Figure()

      fig.add_trace(go.Scatter(x=model_history.epoch, y=loss,
                          mode='lines+markers',
                          name='Loss'))

      fig.add_trace(go.Scatter(x=history.epoch, y=val_loss,
                          mode='lines+markers',
                          name='Validation Loss'))

      fig.update_layout(title = '<b>Training and Validation Loss',
                        title_x=0.5, 
                        plot_bgcolor='white', 
                        font=dict(size=10, color="grey"), 
                        xaxis_title="Epochs",
                        yaxis_title="Loss Value")

      # Modify x-axis
      fig.update_xaxes(
          mirror=True,
          ticks='outside',
          showline=True,
          linecolor='lightgrey',
          gridcolor='white', 
          color='grey')

      # Modify y-axis
      fig.update_yaxes(
          mirror=True,
          ticks='outside',
          showline=True,
          linecolor='lightgrey',
          gridcolor='lightgrey',
          color='grey')

      fig.show()

    '''
      Plot the UNET model's accuracy performance

      @param model_history - the unet model history
    '''
    def plot_accuracy_performance_charts(self, model_history):
      accuracy = model_history.history['accuracy']
      val_accuracy = model_history.history['val_accuracy']

      plt.figure()
      plt.plot(model_history.epoch, accuracy, 'r', label='Training Accuracy')
      plt.plot(model_history.epoch, val_accuracy, 'bo', label='Validation Accuracy')
      plt.title('Training and Validation Accuracy')
      plt.xlabel('Epoch')
      plt.ylabel('Accuracy Value')
      plt.ylim([0.5, 1])
      plt.legend()
      plt.show()

    '''
      Plot the UNET model's accuracy performance - with ploty

      @param model_history - the unet model history
    '''
    def plot_accuracy_performance_charts_plotly(self, model_history):
      accuracy = model_history.history['accuracy']
      val_accuracy = model_history.history['val_accuracy']

      fig = go.Figure()

      fig.add_trace(go.Scatter(x=model_history.epoch, y=accuracy,
                          mode='lines+markers',
                          name='Accuracy'))

      fig.add_trace(go.Scatter(x=history.epoch, y=val_accuracy,
                          mode='lines+markers',
                          name='Validation Accuracy'))

      fig.update_layout(title = '<b>Training and Validation Accuracy',
                        title_x=0.5, 
                        plot_bgcolor='white', 
                        font=dict(size=10, color="grey"), 
                        xaxis_title="Epochs",
                        yaxis_title="Accuracy Value")

      # Modify x-axis
      fig.update_xaxes(
          mirror=True,
          ticks='outside',
          showline=True,
          linecolor='lightgrey',
          gridcolor='white', 
          color='grey')

      # Modify y-axis
      fig.update_yaxes(
          mirror=True,
          ticks='outside',
          showline=True,
          linecolor='lightgrey',
          gridcolor='lightgrey',
          color='grey')

      fig.show()

    '''
      Display a confusion matrix

      @param y_true - true labels
      @param y_pred - predicted labels
    '''
    def display_confusion_matrix(self, y_true, y_pred):
      self.conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred)

      # plot the confusion matrix
      fig, ax = plt.subplots(figsize=(7.5, 7.5))
      ax.matshow(self.conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
      for i in range(self.conf_matrix.shape[0]):
          for j in range(self.conf_matrix.shape[1]):
              ax.text(x=j, y=i,s=self.conf_matrix[i, j], va='center', ha='center', size='xx-large')

      plt.xlabel('Predictions', fontsize=14)
      plt.ylabel('Actuals', fontsize=14)
      plt.title('Confusion Matrix', fontsize=18)
      plt.show()

      # Show the confusion matrix results in a text report
      self._show_confusion_matrix_report()

      '''
        Display the results of the confusion matrix as a text report
      '''
      def _show_confusion_matrix_report(self):
        tn, fp, fn, tp = self.conf_matrix.ravel()

        print("True Positive (TP): ", tp)
        print("True Negative (TN): ", tn)
        print("False Positive (FP): ", fp)
        print("False Negative (FN): ", fn)

        accuracy = (tp + tn) / (tp + tn + fp + fn)

        precision = tp / (tp + fp)

        recall = tp / (tp + fn)

        f1_score = 2 * (precision * recall) / (precision + recall)

        print("\n\nMetrics:")
        print("Accuracy: ", round(accuracy, 2))
        print("Precision: ", round(precision, 2))
        print("Recall: ", round(recall, 2))
        print("F1-score: ", round(f1_score, 2))

In [ ]:
performance_report = PerformanceReport()
performance_report.plot_accuracy_performance_charts_plotly(history)
performance_report.plot_loss_performance_charts_plotly(history)

In [ ]:
print("Accuracy: ", history.history['accuracy'][-1])
print("Loss: ", history.history['loss'][-1])

#### 5.6 Tuning the Model

In [ ]:
unet_model = model_builder.build_fine_tuned_unet_model()

history_1 = unet_model.fit(train_dataset,
               validation_data=validation_dataset,
               steps_per_epoch=STEPS_PER_EPOCH,
               validation_steps=VALIDATION_STEPS,
               epochs=150,
               initial_epoch=50,
               verbose=2)

#### 5.7 Checking the predictions - Tuned Model

In [ ]:
image, mask = next(iter(validation_dataset))
pred = unet_model.predict(image)

image_viewer.show_pred(image, mask, pred)

#### 5.8 Model Performance - Tuned Model

In [ ]:
performance_report.plot_accuracy_performance_charts_plotly(history_1)
performance_report.plot_loss_performance_charts_plotly(history_1)

In [ ]:
print("Accuracy: ", history_1.history['accuracy'][-1])
print("Loss: ", history_1.history['loss'][-1])

### 6. Save the U-Net Model

In [ ]:
# Save the model weights for later use
unet_model.save_weights('../Models/segmentation_model_weights.weights.h5')

In [ ]:
# Save the model for later use
unet_model.save('../Models/segmentation_model.h5')